In [1]:
import re
import string
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [2]:
fake_data = pd.read_csv("D:/Data_Scientist/FakeNews_Detaction/Data/Fake.csv")
true_data = pd.read_csv("D:/Data_Scientist/FakeNews_Detaction/Data/True.csv")

In [3]:
fake_data["class"]=0
true_data["class"]=1

In [4]:
data_fake_manual_testing = fake_data.tail(10)
for i in range(23480, 23470, -1):
    fake_data.drop([i], axis = 0, inplace = True)

data_true_manual_testing = true_data.tail(10)
for i in range(21416, 21406, -1):
    true_data.drop([i], axis = 0, inplace = True)

In [5]:
data_fake_manual_testing.loc[:, "class"]=0
data_true_manual_testing.loc[:, "class"] = 1

C:\Users\MUIZ UD DIN\AppData\Local\Temp\ipykernel_7860\615937241.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing.loc[:, "class"]=0
C:\Users\MUIZ UD DIN\AppData\Local\Temp\ipykernel_7860\615937241.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing.loc[:, "class"] = 1


In [6]:
data_marge = pd.concat([fake_data, true_data], axis=0)

data = data_marge.drop(["title", "subject", "date"],axis=1)

data = data.sample(frac=1)

data.reset_index(inplace=True)
data.drop(["index"], axis=1, inplace=True)

In [7]:
def wordopt(text):
    text = text.lower()
    text = re.sub("\[.*?]", "", text)
    text = re.sub("https?://\S+|www.\.\S+", "", text)
    text = re.sub("<.*>+", "", text)
    text = re.sub("[%s]" % re.escape(string.punctuation), "", text)
    text = re.sub("\w*\d\w*", "", text)
    return text

data["text"] = data["text"].apply(wordopt)

In [8]:
x = data["text"]
y = data["class"]

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [10]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(xv_train, y_train)

pred_lr = LR.predict(xv_test)
LR.score(xv_test, y_test)

0.986096256684492

In [11]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

pred_dt = DT.predict(xv_test)
DT.score(xv_test, y_test)

0.9967023172905526

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
GBC = GradientBoostingClassifier()
GBC.fit(xv_train, y_train)

pred_gb = GBC.predict(xv_test)
GBC.score(xv_test, y_test)

0.9958110516934047

In [13]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier()
RFC.fit(xv_train, y_train)

pred_rf = RFC.predict(xv_test)
RFC.score(xv_test, y_test)

0.984313725490196

In [17]:
def output_label(n):
    if n == 0:
        return "fake news"
    elif n == 1:
        return "not fake news"

def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_label(pred_LR[0]),
                                                                                                             output_label(pred_DT[0]),
                                                                                                             output_label(pred_GBC[0]),
                                                                                                             output_label(pred_RFC[0])))

In [ ]:
news = str(input())
manual_testing(news)